In [3]:
"""
Traversal Methods:
1. Random traversal of nearest neighbors
    Pros:
        no training time
    Cons:
        Low efficiency: ~25% fuel left

Optimizations:
1. Reduced potential breweries to max round trip
2. Neighbor list precompute (memory requirements O(n^2))
"""

'\nTraversal Methods:\n1. Random traversal of nearest neighbors\n    Pros:\n        no training time\n    Cons:\n        Low efficiency: ~25% fuel left\n\nOptimizations:\n1. Reduced potential breweries to max round trip\n2. Neighbor list precompute (memory requirements O(n^2))\n'

In [4]:
import pandas as pd
import numpy as np
import haversine
import random
import timeit
from IPython.display import clear_output

In [5]:
beers = pd.read_csv("beers.csv")
breweries = pd.read_csv("breweries.csv")
categories = pd.read_csv("categories.csv")
geocodes = pd.read_csv("geocodes.csv")
geocodes = geocodes.set_index('id')
styles = pd.read_csv("styles.csv")
coords = geocodes[['latitude', 'longitude']]
# Add home location
home = pd.Series({'latitude': 51.355468, 'longitude': 11.100790 }, name=0)
coords = coords.append(home)
coords = coords.sort_index()

def get_coords(a):
    lat = coords.loc[a]['latitude']
    lon = coords.loc[a]['longitude']
    return (lat, lon)

def dist_coord(x,y):
    h = haversine.haversine(x, y)
    return h

def dist_id(x, y):
    return dist_coord(get_coords(x), get_coords(y))

def get_neighbors(idx):
    a = pd.DataFrame(coords.apply(lambda x: dist_coord(coords.loc[idx], x), axis=1).sort_values())
    a.columns = ["distance to id %d" % idx]
    return a

In [6]:
# reduce candidates to 1/2 fuel
neigh = get_neighbors(0)
a = neigh[get_neighbors(0) < 1000]
a = a.dropna()
ids = a.index
coords = coords.loc[ids]

In [7]:
# make neighbor list
t = timeit.default_timer()
max_neighbors = len(coords)
def n_nearest_neigh(idx, n = max_neighbors):
    return list(get_neighbors(idx).index[1:n+1])

coord_idx = (list(coords.index))
coord_idx
neigh_list = []
for idxs in coord_idx:
    #clear_output()
    #print("Computing neighbor list for id %d" % idxs)
    neigh_list.append((n_nearest_neigh(idxs)))

t = timeit.default_timer() - t
print("Time to compute neighbor list: %fs" % t)

Time to compute neighbor list: 25.802298s


In [8]:
ndf = pd.DataFrame(neigh_list)
ndf.index = coord_idx
coords['neighbors'] = neigh_list
coords
mem = len(coords) ** 2 * 8 / 1000
print("Mem Req for neighbor list %dkB" % mem)

Mem Req for neighbor list 739kB


In [9]:
def get_next_id(curr, visited):
    neigh = coords.loc[curr]['neighbors'] # neighbor list for current id
    try_count = 0     
    
    # this is where decision happens
    radius = 3 # start with this sized halo
    while(True):
        try_count+=1
        next_id = neigh[random.randint(0,radius-1)]
        if (next_id not in set(visited)): return next_id
        # number of tries scaling with radius
        if (try_count > radius**2): radius+=1
            
    return next_id

def fly(start=0, max_dist = 2000):
    visited = []
    curr = start
    travelled = 0
    dist_to_start = 0
    while(True):
        next_id = get_next_id(curr, visited)
        dist_to_next = dist_id(curr, next_id)
        if (dist_to_next + travelled + dist_to_start < max_dist): 
            travelled += dist_to_next
            visited.append(curr)
            dist_to_start = dist_id(start, next_id)
            #print("Current ID: %d Next ID: %d Distance Travelled: %f" %(curr, next_id, travelled))
            curr = next_id
        else:
            return visited, travelled

In [14]:
max_visited = 0
best_res = ""
t_run = timeit.default_timer()
fly_data = []
for i in range(10000):
    t = timeit.default_timer()
    visited, km = fly()
    fly_data.append([visited, km])
    t = timeit.default_timer() - t
    res = "Max: %d Visited %d, Travelled: %f Done in %fs" %(max_visited, len(visited), km, t)
    if (i % 10 == 0):
        clear_output()
        print(res)
    if len(visited) > max_visited:
        max_visited = len(visited)
        best_res = res
        
t_run = timeit.default_timer() - t_run
print(best_res)
print(t_run)

Max: 102 Visited 88, Travelled: 1381.279775 Done in 0.257017s


KeyboardInterrupt: 

In [18]:
fly_df = pd.DataFrame(fly_data)
with open('fly_data.csv', 'a') as f:
    fly_df.to_csv(f, header=False)